# Simulating mating

Mating regimes (`xftsim.mate.MatingRegime`) serve two primary functions:

 1. the mates per female the number and offspring count / sex per mating
 2. deciding who mates with whom

 
## Mating and offspring counts

The first (the mates per female the number and offspring count / sex per mating) is determined by the following arguments to the `MatingRegime` constructor:

 - `mates_per_female` - the number of mating partners per female
 - `offspring_per_pair` - the number of offspring per mate pair
 - `female_offspring_per_pair` - the number of female offspring per mate pair
 
Each of this can be constant or variable.

### Variable counts

`xftsim.utils` provides the `VariableCount` class for random counts which include a `draw` method for generating random variates,  an `expectation` property that returns the expected value, and a `nonzero_fraction` property that returns the expected number of nonzero counts. The later is useful for determining, e.g., the fraction of couples expected to produce offspring. 

The simplest `VariableCount` subclass is the `ConstantCount`:

In [1]:
import xftsim as xft

ccount = xft.utils.ConstantCount(2)
ccount.draw(3), ccount.expectation, ccount.nonzero_fraction

(array([2, 2, 2]), 2, 1.0)

Other useful `VariableCount` subclasses are demonstrated below:

In [2]:
pcount = xft.utils.PoissonCount(2)
nbcount = xft.utils.NegativeBinomialCount(2,.5)
ztpcount = xft.utils.ZeroTruncatedPoissonCount(2)
mixcount = xft.utils.MixtureCount(componentCounts=[xft.utils.ConstantCount(0),
                                                   xft.utils.ConstantCount(3)],
                                  mixture_probabilities= [.4,.6])

for count in [pcount, nbcount, ztpcount, mixcount]:
    print(count.expectation, count.nonzero_fraction)

2 0.8646647167633873
2.0 0.75
2.3130352854993315 1
1.7999999999999998 0.6


 
## Who with whom?

The second (who mates with whom) is mostly determined by the `.mate()` method of the `MatingRegime` but also by two boolean flags provided to the `MatingRegime` constructor:

 - `sex_aware` if `True`, enforces a regime where males may only mate with females and vice versa. In many cases, we have no interest in sex effects and it is therefore convenient to set this to `False` to increase the effective population size.
 
 - `exhaustive` if `False` when assigning mates, male mates are sampled with replacement. This too is convenient to set to `False` but should have little impact either way in sufficiently large simulations.
 
The `mate()` methods are specific each `MatingRegime` subclass. Generally speaking, they map haplotype and phenotype arrays to a `MateAssignment` object. 

In what follows, we first introduce simple random mating regimes, then  `MateAssignment` objects, and finally we'll move on to more complex mating regimes.

Throughout, we'll reference the toy dataset below:

In [3]:
import xftsim as xft

demo = xft.sim.DemoSimulation('BGRM')
demo

<DemoSimulation>
Bivariate GCTA with balanced random mating demo

n = 2000; m = 400
Two phenotypes, height and bone mineral denisty (BMD)
assuming bivariate GCTA infinitessimal archtecture
with h2 values set to 0.5 and 0.4 for height and BMD
respectively and a genetic effect correlation of 0.0.

## Random mating

The simplest mating regime is that of random mating.

In [4]:
from xftsim.mate import RandomMatingRegime
from xftsim.utils import ConstantCount, PoissonCount

rm_regime = RandomMatingRegime(offspring_per_pair = ConstantCount(2),
                                mates_per_female = ConstantCount(1),
                                female_offspring_per_pair = 'balanced',
                                sex_aware = False,
                                exhaustive = True)
                                        

Setting `female_offspring_per_pair` to `"balanced"` rather than a `VariableCount` object will result in a equal number of male and female offspring. 

It is sometimes useful to predict how the population size will change across generations. The `MatingRegime.population_growth_factor` property reveals the expected multiplicative change in population size each generation.

As we specified `rm_regime` such that each female is paired with exactly one male and each mating will produce exactly one offspring, we expect constant population size:

In [5]:
rm_regime.population_growth_factor

1.0

Of course, this is not always the case, as we demonstrate below:

In [6]:
rm_regime2 = RandomMatingRegime(offspring_per_pair = PoissonCount(2.2),
                                mates_per_female = PoissonCount(1.1),
                                female_offspring_per_pair = 'balanced',
                                sex_aware = False,
                                exhaustive = True)
              
rm_regime2.population_growth_factor

1.2100000000000002

## Mate assignments

Mating regime objects map haplotypes and phenotypes to a `MateAssignment` object, which stores information about who mates with whom and the offspring such matings produce:

:::{note}

In practice, users will rarely call `MateAssignment.mate()` directly as this occurs automatically when running a simulation.

:::


In [7]:
mate_assignment = rm_regime.mate(haplotypes=demo.haplotypes,
                                 phenotypes=demo.phenotypes)

:::{note}

Since the mating regime we're using is random, the only information the `mate` method uses from either the haplotypes or phenotypes would be the associated `index.SampleIndex` object.

:::

Two useful perspectives for viewing a `MateAssignment` are the `mating_frame`, which is indexed by couple, and the `reproduction_frame`, which is indexed by offspring:

In [8]:
mate_assignment.get_mating_frame()

,maternal_sample,maternal_iid,maternal_fid,maternal_sex,paternal_sample,paternal_iid,paternal_fid,paternal_sex,n_offspring,n_female_spring
0,0..1_182.1_91,1_182,1_91,0,0..1_3033.1_1516,1_3033,1_1516,0,2,0
1,0..1_2275.1_1137,1_2275,1_1137,1,0..1_398.1_199,1_398,1_199,0,2,2
2,0..1_2040.1_1020,1_2040,1_1020,0,0..1_3189.1_1594,1_3189,1_1594,0,2,2
3,0..1_2345.1_1172,1_2345,1_1172,1,0..1_796.1_398,1_796,1_398,1,2,1
4,0..1_1687.1_843,1_1687,1_843,1,0..1_2154.1_1077,1_2154,1_1077,0,2,1
...,...,...,...,...,...,...,...,...,...,...
1995,0..1_3329.1_1664,1_3329,1_1664,1,0..1_1168.1_584,1_1168,1_584,1,2,0
1996,0..1_920.1_460,1_920,1_460,0,0..1_2262.1_1131,1_2262,1_1131,0,2,2
1997,0..1_877.1_438,1_877,1_438,0,0..1_1204.1_602,1_1204,1_602,1,2,2
1998,0..1_2531.1_1265,1_2531,1_1265,1,0..1_2171.1_1085,1_2171,1_1085,1,2,2


In [9]:
mate_assignment.get_reproduction_frame()

,iid,fid,sex,maternal_sample,maternal_iid,maternal_fid,maternal_sex,paternal_sample,paternal_iid,paternal_fid,paternal_sex
sample,,,,,,,,,,,
0..2_0.2_0,2_0,2_0,1,0..1_182.1_91,1_182,1_91,0,0..1_3033.1_1516,1_3033,1_1516,0
0..2_1.2_0,2_1,2_0,1,0..1_182.1_91,1_182,1_91,0,0..1_3033.1_1516,1_3033,1_1516,0
0..2_2.2_1,2_2,2_1,0,0..1_2275.1_1137,1_2275,1_1137,1,0..1_398.1_199,1_398,1_199,0
0..2_3.2_1,2_3,2_1,0,0..1_2275.1_1137,1_2275,1_1137,1,0..1_398.1_199,1_398,1_199,0
0..2_4.2_2,2_4,2_2,0,0..1_2040.1_1020,1_2040,1_1020,0,0..1_3189.1_1594,1_3189,1_1594,0
...,...,...,...,...,...,...,...,...,...,...,...
0..2_3995.2_1997,2_3995,2_1997,0,0..1_877.1_438,1_877,1_438,0,0..1_1204.1_602,1_1204,1_602,1
0..2_3996.2_1998,2_3996,2_1998,0,0..1_2531.1_1265,1_2531,1_1265,1,0..1_2171.1_1085,1_2171,1_1085,1
0..2_3997.2_1998,2_3997,2_1998,0,0..1_2531.1_1265,1_2531,1_1265,1,0..1_2171.1_1085,1_2171,1_1085,1


## Nonrandom mating regimes


:::{warning}

It will be challenging to to understand this information if you haven't read [the tutorial on indexing](./indexing.ipynb)! Specifically, understanding `xftsim.index.ComponentIndex` is essential.

:::

As `xftsim` mating regimes are privy to haplotypes and phenotypes, the can be very general. Here we present a few commonly used regimes.

### Linear assortative mating

By "linear assortative mating", we refer to mating mediated through a linear combination of phenotypic components. An exchangeable linear regime is provided by `xft.mate.LinearAssortativeMatingRegime`. Concretely, assume we want an exchangeable regime for $k$ phenotypes with cross-mate correlations fixed at `r`. That is, given female and male phenotypes $Y,\tilde{Y}\in R^{n\times k}$, respectively, this regime will generate a mating permutation $P$

such that the empirical correlation is
$$\text{corr}(Y,P\tilde{Y})\approx r\cdot1_{k\times k}.$$


### Univariate primary phenotypic assortment

We can implement univariate assortative mating mediated through a single phenotype via `LinearAssortativeMatingRegime`. Below we do this for height, setting the cross-mate correlation to 0.5. We iterate for a single generation and then observe the sample mating statstics:

In [10]:
from xftsim.mate import LinearAssortativeMatingRegime

reg_pp = LinearAssortativeMatingRegime(r=.5,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name='height', 
                                                          component_name='phenotype'))

demo_pp = xft.sim.DemoSimulation()
demo_pp.mating_regime = reg_pp
demo_pp.run(1)
xmate_corr = demo_pp.results['mating_statistics']['mate_correlations']
xmate_corr.iloc[:xmate_corr.shape[0]//2,xmate_corr.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.242735,-0.003983,0.229452,0.012198,0.334080,0.006560
BMD.additiveGenetic.mother,-0.006012,0.011312,-0.009585,0.023872,-0.010987,0.025515
height.additiveNoise.mother,0.283236,-0.017278,0.248033,0.007626,0.376128,-0.005711
BMD.additiveNoise.mother,0.005301,-0.011793,-0.000940,0.022259,0.003159,0.008969
height.phenotype.mother,0.373033,-0.015124,0.338575,0.014036,0.503650,0.000554
BMD.phenotype.mother,0.000195,-0.001742,-0.006786,0.031953,-0.004576,0.022950


### Univariate social / genetic homogamy

Social or genetic homogamy can be modeled by specifying that mating operates only on the non-heritable or heritable components respectively:


In [11]:
from xftsim.mate import LinearAssortativeMatingRegime

reg_social = LinearAssortativeMatingRegime(r=.5,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name='height', 
                                                          component_name='additiveNoise'))

reg_genetic = LinearAssortativeMatingRegime(r=.5,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name='height', 
                                                          component_name='additiveGenetic'))


demo_soc = xft.sim.DemoSimulation(); demo_gen = xft.sim.DemoSimulation()
demo_soc.mating_regime = reg_social; demo_gen.mating_regime = reg_genetic; 
demo_soc.run(1); demo_gen.run(1)
xmate_corr_soc = demo_soc.results['mating_statistics']['mate_correlations']
xmate_corr_soc.iloc[:xmate_corr_soc.shape[0]//2,xmate_corr_soc.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.013878,-0.032360,-0.014410,0.028975,-0.000577,0.000741
BMD.additiveGenetic.mother,0.029287,0.014812,0.005467,-0.014103,0.024533,-0.000973
height.additiveNoise.mother,0.004405,-0.010921,0.508129,-0.014149,0.367805,-0.017805
BMD.additiveNoise.mother,0.021196,0.009432,0.014848,-0.002474,0.025572,0.004290
height.phenotype.mother,0.012626,-0.029914,0.360482,0.009350,0.267618,-0.012471
BMD.phenotype.mother,0.035198,0.016823,0.014845,-0.011075,0.035423,0.002625


In [12]:
xmate_corr_gen = demo_gen.results['mating_statistics']['mate_correlations']
xmate_corr_gen.iloc[:xmate_corr_gen.shape[0]//2,xmate_corr_gen.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.507781,0.034394,-0.000434,0.002611,0.370425,0.024729
BMD.additiveGenetic.mother,0.021875,0.007018,0.012762,0.006168,0.024902,0.009160
height.additiveNoise.mother,-0.029332,-0.006682,0.020333,-0.004088,-0.007184,-0.007418
BMD.additiveNoise.mother,-0.017322,0.003566,0.019329,0.006404,0.000881,0.007042
height.phenotype.mother,0.350034,0.020425,0.013906,-0.000954,0.265291,0.012857
BMD.phenotype.mother,0.002095,0.007431,0.023026,0.008952,0.017644,0.011467


### Exchangeable bivariate phenotypic assortment

Exchangeable bivariate phenotypic assortment is also easily accommodated in this framework. Here we implement such a regime for $r$=.1 across height and bone mineral density (BMD):

In [13]:
reg_biv = LinearAssortativeMatingRegime(r=.1,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name=['height','BMD'], 
                                                          component_name='phenotype'))

demo_biv = xft.sim.DemoSimulation()
demo_biv.mating_regime = reg_biv
demo_biv.run(1)
xmate_corr = demo_biv.results['mating_statistics']['mate_correlations']
xmate_corr.iloc[:xmate_corr.shape[0]//2,xmate_corr.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.033304,0.049230,0.058150,0.048000,0.065181,0.068972
BMD.additiveGenetic.mother,0.054839,0.064500,0.020884,0.054396,0.052817,0.083750
height.additiveNoise.mother,0.055989,0.050736,0.061472,0.053519,0.083158,0.074237
BMD.additiveNoise.mother,0.037729,0.060261,0.041121,0.053044,0.055817,0.079977
height.phenotype.mother,0.062385,0.069489,0.083174,0.070627,0.103346,0.099594
BMD.phenotype.mother,0.064538,0.088454,0.045707,0.076353,0.077543,0.116220


## Generalized assortive mating

:::{note}

Coming soon

:::

## Batched mating regimes


:::{note}

Coming soon

:::
